# 0) Download Required Libraries

In [1]:
!pip install transformers
import transformers
import numpy as np
import random
import re
from math import *


# 1) Initialize parameters

In [2]:
DEBUG=False


senderGeneratedText="حققت تونس الدولة المضيفة تعادلا الإيجابي الاثنين أمس الجمعة إلى صفر صفرا مع اليونان الأوروبية الجنسية الأمريكية الشرقية مقابل اليابان المصرية الرسمية الجنسية الداخلية الألمانية"
senderGeneratedText="أعلن الريس على عن حياته قريبا السبت الجمعة الماضي إلى حين تاريخ النهاية مع العدالة المتحدة الجمعة قبل الغد وبعد الأربعاء هذا هذان هذانث هذانثة هذانثةه هو"


from enum import Enum
class SourceCoding(Enum):
    HUFFMAN = 1
    BINARY  = 3

class TextGenerationLang(Enum):
    Arabic  = 1

class ModelName(Enum):
    Roberta = 7

class StegoPriority(Enum):
    Capacity=1
    Efficiency=2
    Imperceptibility=3

top_k=32
sourceCodingType=SourceCoding.HUFFMAN
TextGenerationLang=TextGenerationLang.Arabic
choosenModel=ModelName.Roberta
choosenStegoPriority=StegoPriority.Capacity

# Discard the first ? words because they were added to control the context and do not have any data hidden inside them.
# This is a constant number that should match between sender and receiver before lunch. (Same as length of senderGeneratedText variable in Sender)
numberOfContextWordAdded=4

# 3) Models

## You Model

In [3]:
if choosenModel==ModelName.Roberta:
  from transformers import pipeline,TFAutoModelForMaskedLM,TFBertForMaskedLM
  from transformers import AutoTokenizer

  # For example, this model is not good; you should use your trained model instead.
  checkpoint="xlm-roberta-base"

  tokenizer = AutoTokenizer.from_pretrained(checkpoint)
  model_name=checkpoint

  def fillArabicMaskWithRoberta(text,top_k=30):
    text=text+" <mask>"
    model = pipeline(task ="fill-mask",model=model_name,tokenizer=tokenizer,top_k =top_k,device=0)
    return model(text)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

# 4) Enconding

## 1) Huffman Coding

In [4]:
# A Huffman Tree Node
class Node:
    def __init__(self, prob, symbol, left=None, right=None):
        # probability of symbol
        self.prob = prob

        # symbol
        self.symbol = symbol

        # left node
        self.left = left

        # right node
        self.right = right

        # tree direction (0/1)
        self.code = ''

""" A helper function to print the codes of symbols by traveling Huffman Tree"""
codes = dict()

def Calculate_Codes(node, val=''):
    # huffman code for current node
    newVal = val + str(node.code)

    if(node.left):
        Calculate_Codes(node.left, newVal)
    if(node.right):
        Calculate_Codes(node.right, newVal)

    if(not node.left and not node.right):
        codes[node.symbol] = newVal

    return codes

""" A helper function to calculate the probabilities of symbols in given data"""
def Calculate_Probability(data):
    symbols = dict()
    for element in data:
        if symbols.get(element) == None:
            symbols[element] = 1
        else:
            symbols[element] += 1
    return symbols

""" A helper function to obtain the encoded output"""
def Output_Encoded(data, coding):
    encoding_output = []
    for c in data:
      #  print(coding[c], end = '')
        encoding_output.append(coding[c])

    string = ''.join([str(item) for item in encoding_output])
    return string

""" A helper function to calculate the space difference between compressed and non compressed data"""
def Total_Gain(data, coding):
    before_compression = len(data) * 8 # total bit space to stor the data before compression
    after_compression = 0
    symbols = coding.keys()
    for symbol in symbols:
        count = data.count(symbol)
        after_compression += count * len(coding[symbol]) #calculate how many bit is required for that symbol in total
    if DEBUG:
      print("Space usage before compression (in bits):", before_compression)
      print("Space usage after compression (in bits):",  after_compression)

def Huffman_Encoding(data):
    symbol_with_probs = Calculate_Probability(data)
    symbols = symbol_with_probs.keys()
    probabilities = symbol_with_probs.values()
    if DEBUG:
      print("symbols: ", symbols)
      print("probabilities: ", probabilities)


    nodes = []
    # converting symbols and probabilities into huffman tree nodes
    for symbol in symbols:
        nodes.append(Node(symbol_with_probs.get(symbol), symbol))

    while len(nodes) > 1:
        # sort all the nodes in ascending order based on their probability
        nodes = sorted(nodes, key=lambda x: x.prob)
        # for node in nodes:
        #      print(node.symbol, node.prob)

        # pick 2 smallest nodes
        right = nodes[0]
        left = nodes[1]

        left.code = 0
        right.code = 1

        # combine the 2 smallest nodes to create new node
        newNode = Node(left.prob+right.prob, left.symbol+right.symbol, left, right)

        nodes.remove(left)
        nodes.remove(right)
        nodes.append(newNode)

    huffman_encoding = Calculate_Codes(nodes[0])
    if DEBUG:
      print("symbols with codes", huffman_encoding)

    Total_Gain(data, huffman_encoding)
    encoded_output = Output_Encoded(data,huffman_encoding)
    return encoded_output, nodes[0],huffman_encoding


def Huffman_Decoding(encoded_data, huffman_tree,wordSplit=False):
    tree_head = huffman_tree
    decoded_output = []
    for x in encoded_data:
        if x == '1':
            huffman_tree = huffman_tree.right
        elif x == '0':
            huffman_tree = huffman_tree.left
        try:
            if huffman_tree.left.symbol == None and huffman_tree.right.symbol == None:
                pass
        except AttributeError:
            decoded_output.append(huffman_tree.symbol)
            huffman_tree = tree_head

    delimiter= ' ' if wordSplit else ''
    string = delimiter.join([str(item) for item in decoded_output])
    return string





In [5]:

def encodeByHuffman(data,wordSplit=False):
  data=data.strip()
  if wordSplit:
    data=data.split(' ')

  global codes
  codes=dict()
  encoding, tree,mapping = Huffman_Encoding(data)
  return (encoding,tree,mapping)

def decodeByHuffman(encoding,tree,wordSplit=False):
  return Huffman_Decoding(encoding,tree,wordSplit)



## 2) Binary Tree Coding

In [6]:
def buildBinaryTreeCoding(words,currCode=''):

  l=len(words)

  if log2(l)!= log2(pow(2,int(log2(l)))):
    print('The Length Is Not Power Of 2')
    return None

  if l==1:
    return [currCode]
    # return [(words[0],currCode)]

  mid=int(l/2)
  a=buildBinaryTreeCoding(words[:mid],currCode+'0')
  b=buildBinaryTreeCoding(words[mid:],currCode+'1')

  return a+b

def binaryTreeEncoding(words):
  l=len(words)
  leafNb=int(pow(2,int(log2(l))))
  restToCeilLog=int(pow(2,ceil(log2(l))))-leafNb
  words=words+['NO_TOKEN_'+str(i) for i in range(1,restToCeilLog)]
  if DEBUG==True:
    print(len(words))
    print(words)
  wordsCode=buildBinaryTreeCoding(words)
  mapWordToCode= {k:v  for k,v in zip(words,wordsCode) }
  encoding=''.join(list(map(lambda x:mapWordToCode[x],words)))
  return encoding,mapWordToCode



In [7]:
def encodeByBinaryTree(data,wordSplit=False):
  data=data.strip()
  if wordSplit:
    data=data.split(' ')
  else:
    data=list(set(data))

  encoding,mapping = binaryTreeEncoding(data)
  return (encoding,mapping)


# OO ) Main

### Common Functions

In [8]:
def fillMask(generatedText,top_k=top_k):
  top_k=top_k*2
  masks=[]
  if choosenModel==ModelName.Roberta:
    masks=fillArabicMaskWithRoberta(generatedText,top_k=top_k)
  return masks

In [9]:
def generateNextMaskedWord(generatedText,wordIdx,sourceCodingType,top_kth):
  tok_k=top_kth

  if TextGenerationLang==TextGenerationLang.Arabic:
    substitution=r'[^\u0620-\u064a ]*'
  elif TextGenerationLang==TextGenerationLang.English:
    substitution=r'[^a-zA-Z ]*'

  masks=fillMask(generatedText,top_k=top_k)
  masks=list(map(lambda ele: re.sub(substitution, '', ele["sequence"].split(" ")[-1]),masks))

  if DEBUG==True:
    print("Masks: ",masks)

  lastWord=''
  if DEBUG==True:
    print("last element of genereted text : " )
    print(generatedText.split(" "))

  if len(generatedText.split(" "))>0:
    if DEBUG==True:
      print("last element of genereted text : "+ generatedText.split(" ")[-1])
    lastWord=generatedText.split(" ")[-1]
  masks=list(filter(lambda ele: (ele!='' and len(ele)!=1 and ele!=lastWord) ,masks))
  masks=masks[0:max(len(masks),int(top_k/2))]

  encoding=mapWordToCode=None
  if sourceCodingType==SourceCoding.HUFFMAN:
    masks=' '.join(masks)
    encoding,_,mapWordToCode=encodeByHuffman(masks,True)
  elif sourceCodingType==SourceCoding.BINARY:
    l=len(masks)
    leafNb=int(pow(2,int(log2(l))))
    if DEBUG==True:
      print('leafNb: ',leafNb)
    masks=masks[:leafNb]
    # if DEBUG==True:
    print("edit len mask : ",len(masks))
    masks=' '.join(masks)
    encoding,mapWordToCode=encodeByBinaryTree(masks,True)

  mapCodeToWord=dict(map(reversed, mapWordToCode.items()))
  return (encoding,mapWordToCode,mapCodeToWord)

## Receiver

### Masked Generation

In [12]:
senderGeneratedTextList=senderGeneratedText.split(" ")

receiverGeneratedText=senderGeneratedTextList[:numberOfContextWordAdded]
senderGeneratedTextList=senderGeneratedTextList[numberOfContextWordAdded:]

if DEBUG==True:
  print(receiverGeneratedText)
  print(senderGeneratedTextList)

In [13]:
if DEBUG==True:
  print(senderGeneratedText)
END_TOKEN_CODE="0"*8

senderGeneratedTextList=senderGeneratedText.split(" ")
receiverGeneratedText=' '.join(senderGeneratedTextList[:numberOfContextWordAdded])
senderGeneratedTextList=senderGeneratedTextList[numberOfContextWordAdded:]

receiverEncodingSecretMessage=""

for i in range(0,len(senderGeneratedTextList)):
  encoding,mapWordToCode,mapCodeToWord=generateNextMaskedWord(receiverGeneratedText,i,sourceCodingType,top_k)
  if DEBUG==True:
    print(mapCodeToWord)
  currSenderWord=senderGeneratedTextList[i]
  currSenderCode=mapWordToCode.get(currSenderWord)

  if DEBUG==True:
    print(currSenderWord)
    print(currSenderCode)

  if len(receiverGeneratedText)!=0:
    receiverGeneratedText=receiverGeneratedText+" "
  receiverGeneratedText=receiverGeneratedText+currSenderWord
  receiverEncodingSecretMessage=receiverEncodingSecretMessage+currSenderCode

if DEBUG==True:
  print("receiverGeneratedText : "+receiverGeneratedText)
  print("length of secret message enconding : "+str(len(receiverEncodingSecretMessage)))
  print(receiverEncodingSecretMessage)



receiverEncodingSecretMessage=receiverEncodingSecretMessage[::-1]

x = re.search(END_TOKEN_CODE, receiverEncodingSecretMessage)

receiverEncodingSecretMessage=receiverEncodingSecretMessage[x.end():]
receiverEncodingSecretMessage=receiverEncodingSecretMessage[::-1]

if DEBUG==True:
  print(receiverEncodingSecretMessage)



Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertFor

1010000010000011001001001111110100110010011111110100000111011010100110001111010000011101101001010100110001


In [14]:
encodingSecretMessage=receiverEncodingSecretMessage
decodingSecretMessage=""
mapCodeToLetter={'0000': 'ل', '000100': 'ص', '0001010': 'ج', '0001011': 'ز', '000110': 'أ', '000111': 'ف', '001000': 'ك', '00100100': 'إ', '0010010100': 'ظ', '0010010101': 'آ', '001001011': 'ء', '0010011': 'ش', '00101': 'ب', '00110': 'و', '0011100': 'ح', '0011101': 'خ', '001111': 'د', '01000': 'م', '01001': 'ع', '0101000': 'ط', '0101001': 'ذ', '0101010': 'ى', '0101011': 'ض', '01011': 'ر', '0110': 'ي', '011100': 'ة', '011101': 'س', '01111': 'ت', '100000': 'ق', '10000100': 'ث', '10000101': 'غ', '1000011': 'ئ', '10001': 'ه', '1001': 'ن', '101': 'ا', '11': ' '}

encodingSecretMessageLength=len(encodingSecretMessage)
while len(encodingSecretMessage)!=0:
  for i in range(0,encodingSecretMessageLength):
    senderCurrEncodingMatchTest=encodingSecretMessage[0:encodingSecretMessageLength-i]
    if mapCodeToLetter.get(senderCurrEncodingMatchTest)==None:
      continue
    senderChoosenLetter=mapCodeToLetter.get(senderCurrEncodingMatchTest)

    decodingSecretMessage=decodingSecretMessage+senderChoosenLetter
    encodingSecretMessage=encodingSecretMessage[encodingSecretMessageLength-i:]
    break


print(decodingSecretMessage)


الموعد عند الساعه السابعه
